In [2]:
# ----------------------
# 코드 버전 
# ----------------------

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
import copy  # deepcopy를 위해 추가

# ----------------------
# 1. 변수 및 입력 값 설정
# ----------------------

# 입력 값 설정
initial_price = 0.0001  # 초기 가격
slope = 0.000001  # 발행량 기울기
supply_exponent = 3  # 발행량의 제곱근
weight_log_multiplier = 32.5  # 가중치 배수
weight_log_denominator = 100_000_000  # 가중치 로그 내 분모
eth_price_usd = 4000  # 현재 ETH 가격 ($)
x_percentage = 5  # 분석 구간 퍼센티지
exchange_rate = 1400  # 환율 1달러 = 1400원

# 고정 값 설정
initial_spread = 0.9  # 초기 스프레드
final_spread = 0.1  # 최종 스프레드
eth_per_unit = 5  # 단위별 유입 ETH 양
max_supply = 100_000_000  # 최대 발행량 목표

# ----------------------
# 2. 본딩 커브 데이터 생성
# ----------------------

def calculate_bonding_curve(initial_price, slope, supply_exponent, weight_log_multiplier, 
                            weight_log_denominator, max_supply, eth_per_unit, initial_spread, final_spread):
    """
    본딩 커브와 관련된 값들을 계산하는 함수
    """
    dynamic_cumulative_supply = 0
    current_price = initial_price
    dynamic_n = 0
    weight = 0

    # 최대 발행량 도달 시점 기록 변수
    max_supply_reached_n = None

    # 목표 발행량에 도달할 때까지 n 계산
    while dynamic_cumulative_supply < max_supply:
        weight = weight_log_multiplier * np.log1p(dynamic_cumulative_supply / weight_log_denominator)
        current_price = initial_price + slope * weight * (dynamic_cumulative_supply ** (1 / supply_exponent))
        issued_supply = eth_per_unit / current_price
        dynamic_cumulative_supply += issued_supply

        # 최대 발행량에 도달하면 기록
        if dynamic_cumulative_supply >= max_supply and max_supply_reached_n is None:
            max_supply_reached_n = dynamic_n
        dynamic_n += 1

    spread_decrement = (initial_spread - final_spread) / max_supply_reached_n

    # 테이블 생성
    data = []
    cumulative_supply = 0
    spread = None
    current_price = initial_price
    sell_price = None
    weight = None
    area_buy_eth = 0
    area_sell_eth = 0

    for n in range(max_supply_reached_n + 1):
        if n == 0:
            spread = initial_spread
            data.append({
                "Unit (n)": n,
                "ETH Input (Cumulative)": 0,
                "Issued Tokens": 0,
                "Cumulative Supply": cumulative_supply,
                "Buy Price (ETH)": current_price,
                "Spread": None,
                "Sell Price (ETH)": None,
                "Weight": None
            })
            continue

        weight = weight_log_multiplier * np.log1p(cumulative_supply / weight_log_denominator)
        current_price = initial_price + slope * weight * (cumulative_supply ** (1 / supply_exponent))
        eth_input = eth_per_unit
        issued_supply = eth_input / current_price
        prev_cumulative_supply = cumulative_supply
        cumulative_supply += issued_supply
        spread = max(final_spread, spread - spread_decrement)
        sell_price = current_price * (1 - spread)

        supply_diff = cumulative_supply - prev_cumulative_supply
        area_buy_eth += current_price * supply_diff
        area_sell_eth += sell_price * supply_diff

        data.append({
            "Unit (n)": n,
            "ETH Input (Cumulative)": eth_input * n,
            "Issued Tokens": issued_supply,
            "Cumulative Supply": cumulative_supply,
            "Buy Price (ETH)": current_price,
            "Spread": spread,
            "Sell Price (ETH)": sell_price,
            "Weight": weight
        })

    area_buy_usd = area_buy_eth * eth_price_usd
    area_sell_usd = area_sell_eth * eth_price_usd

    return data, max_supply_reached_n, area_buy_eth, area_sell_eth, area_buy_usd, area_sell_usd

# 계산 함수 실행
data, max_supply_reached_n, area_buy_eth, area_sell_eth, area_buy_usd, area_sell_usd = calculate_bonding_curve(
    initial_price, slope, supply_exponent, weight_log_multiplier, weight_log_denominator, 
    max_supply, eth_per_unit, initial_spread, final_spread)

# ----------------------
# 3. 데이터 결과 출력
# ----------------------

# 테이블 생성
df = pd.DataFrame(data)
df["Buy Price (ETH)"] = df["Buy Price (ETH)"].map("{:.6e}".format)
df["Sell Price (ETH)"] = df["Sell Price (ETH)"].map("{:.6e}".format)

# CSV 파일로 저장 (전체 데이터 저장)
csv_filename = "bc_table.csv"
df.to_csv(csv_filename, index=False)
print(f"\n### Bonding Curve Table has been saved to '{csv_filename}' with {len(df)} rows.")


### Bonding Curve Table has been saved to 'bc_table.csv' with 100701 rows.
